<a href="https://colab.research.google.com/github/himani5991/coursera--test/blob/master/N_model_dynamics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this part we 
 - encode dynamics of N-model (see description in "Problem formulation (N model).pdf"),
 - find the best threshold policy,
 - code and play with the REINFORCE policy gradient algorithm,
 - fix a policy and implement a value NN approximator,
 - write an actor-critic algorithm.


The dynamics of the "uniformized" N model is provided next.

Class `ProcessingNetwork` provides an interface for N-model formulation. 

Method `Nmodel_from_load` allows to create an N-model instance with a given load. 

Method `next_state_N1` generates the next state of the system given the current state and the action.

In [ ]:
import random as r
import numpy as np
import math

In [ ]:
network_dictionary = {
    'Nmodel': {
        'A': [[-1, 0], [0, -1]],
        'D': [[1, 1]],
        'alpha': [1.3 * 0.95, 0.4 * 0.95],
        'mu': [1, 1. / 2, 1],
        'name': 'N-model',
        'holding_cost': [3., 1.]}
}


class ProcessingNetwork:
    # N-model network class
    def __init__(self, A, D, alpha, mu, holding_cost,  name):
        self.alpha = np.asarray(alpha)  # arrival rates
        self.mu = np.asarray(mu)  # service rates
        self.uniform_rate = np.sum(alpha)+np.sum(mu)  # uniform rate for uniformization
        self.p_arriving = np.divide(self.alpha, self.uniform_rate)# normalized arrival rates
        self.p_compl = np.divide(self.mu, self.uniform_rate) #normalized service rates
        self.cumsum_rates = np.unique(np.cumsum(np.concatenate([self.p_arriving, self.p_compl])))


    @classmethod
    def Nmodel_from_load(cls, load: float):
        # another constructor for the standard queuing networks
        # based on a queuing network name, find the queuing network info in the 'network_dictionary.py'
        return cls(A=network_dictionary['Nmodel']['A'],
                   D=network_dictionary['Nmodel']['D'],
                   alpha=np.asarray([1.3 * load, 0.4 * load]),
                   mu=network_dictionary['Nmodel']['mu'],
                   holding_cost=network_dictionary['Nmodel']['holding_cost'],
                   name=network_dictionary['Nmodel']['name'])

    def next_state_N1(self, state, action):
        """
        :param state: current state
        :param action: [1, 0] if class 1 is prioritized; [0, 1] if class 2 is prioritized.
        :return: next state
        """

        w = np.random.random()
        wi = 0 
        while w > self.cumsum_rates[wi]: #randomly choose which state to go to 
            wi += 1
        if wi == 0:
            state_next = state + np.array([1, 0]) #station 1 new arrival 
        elif wi == 1:
            state_next = state + np.array([0, 1]) #station 2 new arrival
        elif wi == 2 and (state[0] > 0):
            state_next = state - np.array([1, 0]) #station 1 departure 
        elif wi == 3 and ((action[0] == 1 or state[1] == 0) and state[0] > 1):
            state_next = state - np.array([1, 0]) #station 1 departure
        elif wi == 4 and ((action[0] == 0 or state[0] < 2) and state[1] > 0):
            state_next = state - np.array([0, 1]) #station 2 departure 
        else:
            state_next = state #no change 
        return state_next

We define an instance of the N model with load $\rho=0.95$.

In [ ]:
n_model_095 = ProcessingNetwork.Nmodel_from_load(load = 0.95)

An example how next state is generated, given current state and action.

In [ ]:
 # state [1, 1] means there is one class 1 job and one class 2 job in the system; 
 # action [1, 0] means class 1 jobs have priority. Another feasible action is [0, 1] that means that class 2 jobs have priority. 
print(n_model_095.next_state_N1(state = [1, 1], action = [0, 1]))

Let's check that always prioritizing class 1 job is a bad idea. 

In [ ]:
state = np.array([0, 0])
for t in range(10**5):
  state = n_model_095.next_state_N1(state, action = [1, 0])
  print('time step:', t,' ', '# of jobs in the system:', np.sum(state))

The N-model system with load $\rho=0.95$ is difficult to optimize. We decrease the load to make the control problem easier.

In [ ]:
n_model_085 = ProcessingNetwork.Nmodel_from_load(load = 0.85)

# Task 1: threshold policy

Find the best threshold policy for the N-model with load $\rho=0.85$. Server 2 of the N-model operating under the threshold policy gives priority to
class 1 jobs, if the number of class 1 jobs in the system is larger than a fixed threshold $T\in \mathbb{R}$.

- For each fixed $T$, run for at least $10^5$ time-steps, compute the average cost. Recall, one-step cost of state $x = (x_1, x_2)$ is $3x_1+x_2$.

In [ ]:
## your code


The best threshold policy is our benchmark now. We should try a policy-gradient algorithm to find a better policy.

# Task 2: policy gradient implementation.

Algorithm pseudocode:
- Initialize $z_0$, $\hat \eta$, learning rate $\alpha_t$, $\beta>0$
- For each transition step $t=0, ..., T$ do
  - Update average cost estimate
$$
\hat \eta_{t+1} = \hat \eta_t+\beta \alpha_t (c(x_t) - \hat \eta_t)
$$
  - If $x_t = x^*$ then $z_{t+1} = 0$, otherwise 
  $$
   z_{t+1} = z_t +\nabla_\theta \log \pi (a_t|x_t)
  $$
  - Update parameters
  $$\theta_{t+1} = \theta_t - \alpha_t(c(x_t) - \hat \eta_t)z_t$$






Step 1. Define a policy parameterization. A good choice is 
$$
\pi_\theta(\text{priority to class }1| x) = \frac{1}{1+\exp(\theta_1x_1+\theta_2x_2 +b)}, 
$$
where $\theta = (\theta_1, \theta_2)$ are parameters we want to optimize, $b\in \mathbb{R}$.

Hint: $$\pi_\theta(\text{priority to class }1| x) \geq 1/2\quad \text{ if }\quad b \leq -\theta_1x_1-\theta_2x_2.$$

Step 2. We need to be able to compute $\nabla\log \pi_\theta(a|x) = \frac{\nabla\pi_\theta(a|x)}{\pi_\theta(a|x)}$.

Other hints: 
- $\pi_\theta(\text{priority to class }2| x) = 1 - \pi_\theta(\text{priority to class }1| x)$
- $\nabla_\theta \pi_\theta(\text{priority to class }2| x) =  - \nabla_\theta \pi_\theta(\text{priority to class }1| x)$

In [ ]:
## your code

Step 3. Define the recurrent state, intial state, initial average cost, number of steps, etc.

Step 4. Implement the algorithm. Try different parametrizations, initial states, learning rates, etc.




In [ ]:
## your code



# Task 3. Implement "value" neural network for a fixed policy.

Step 1. Read an example of how a feed-forward neural network is created in TensorFlow: https://colab.research.google.com/drive/1xPIZzztD5XG0RcLvsP9Drs3puFCWGAOo?usp=sharing

Step 2. Let's consider the threshold policy $T=5$. Generate $N=5000$ regenerative cycles under this policy. Transitions from $x^*$ to itself counts as cycles. We have trajectory data:
$$
(x_0=x^*, a_0, x_1, a_1, x_2, ...., x_{\sigma_N-1}),
$$
where $\sigma_n$ is the $n$th time the regenerative state $x^*$ is visited.

Step 3. Estimate the average cost. Recall,
$$
\eta = \frac{1}{\mathbb{E}[\sigma(x^*)]} \mathbb{E}\left[\sum\limits_{t=0}^{\sigma(x^*)-1}c(x_t)~|~x_0=x\right].
$$
Hence, its estimate is
$$
\hat \eta = \frac{1}{\sigma_N}\sum\limits_{t=0}^{\sigma_N-1}c(x_t).
$$

Step 4. Compute one-replication estimates of the value function.

The threshold policy $T=5$ has its (relative) value function:
$$
V^{\pi}(x):=\mathbb{E}\left[\sum\limits_{t=0}^{\sigma(x^*)-1} (c(x_t) - \eta^\pi) ~|~x_0=x\right].
$$

We have trajectory data $(x_0, a_1, x_1, a_2, ..., x_{\sigma_N-1})$ generated  under this policy. For each state in this trajectory, we can compute one-replication estimate of the value function. For example, for a state $x_t$ visited at step $t$:
$$
\hat V_t:=\sum\limits_{k=t}^{\sigma(t)-1} (c(x_k) - \hat \eta),
$$
where $\sigma(t) = \min\{k>t~|~x_k=x^*\}$.

We note that the one-replication estimate is computed for every timestep.

Step 5. We define $f_\phi$, $\phi\in \Phi$, as a "value" neural network. We want to find parameters $\phi^*$ s.t. 
$$
\phi^* = \arg\min\limits_{\phi \in \Phi}\sum\limits_{t=0}^{\sigma_N-1} \left(f_\phi(x_t) - \hat V_t\right)^2.
$$

Please, note, there is no need to average one-replication estimates that correspond to one state:
$$
\arg\min\limits_{x\in B} \sum\limits_{i=1}^n\left(x-a_i\right)^2 = \arg\min\limits_{x\in B} \left(x-\frac{1}{n}\sum\limits_{i=1}^na_i\right)^2, 
$$
where $B$ is an arbitrary subset of $\mathbb{R}$.

In [ ]:
threshold = 5
N = 5000

## collect data


  


In [ ]:
## estimate the average cost

In [ ]:
## estimate the value function

In [ ]:
## define NN and optimize its parameters

# Task 4 (Optional).  Implement an actor-critic algorithm using 'value' neural network as a critic. 

See slide 46 for the pseudecode of the algorithm.

In [ ]:
## your code